In [ ]:
import pandas as pd
import numpy as np
import psutil
import matplotlib.pyplot as plt
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from pathlib import Path
import math
import os, sys
from numpy.linalg import pinv
from scipy.stats import f
import re

In [ ]:
path0 = '/mnt/sda1/RA5/data'
path1 = '/mnt/sda1/RA5/intermediate/siyoung/Event_study'

In [ ]:
ds_combined = pd.read_parquet(os.path.join(path1,'ds_combined.parquet'))
ds_combined['months_from_birth'] = ds_combined['months_from_birth'].replace(np.nan, -1)
ds_combined['months_from_birth'] = ds_combined['months_from_birth'].astype(int)

In [ ]:
new_var_name = {
 'shc_weekday_rt':'weekday spending ratio',
 'shc_food_rt' : 'food industry ratio',
 'shc_ent_rt' : 'entertainment ratio',
 'shc_dep_rt' : 'dep store ratio',
 'shc_mart_rt' : 'large store ratio',
 'shc_ssm_rt' : 'small retail ratio',
 'shc_clothes_rt' : 'clothing/accessories ratio',
 'shc_cul_rt' : 'sport/culture/leiture ratio',
 'shc_acco_rt' : 'accomodation ratio',
 'shc_travel_rt' : 'travel ratio',
 'shc_trans_rt' : 'transportation ratio',
 'shc_beauty_rt' : 'beauty ratio',
 'shc_household_rt' : 'household living ratio',
 'shc_edu_rt' : 'education ratio',
 'shc_med_rt' : 'medical ratio',
 'shc_furn_rt' : 'electronics/furniture ratio',
 'shc_car_rt' : 'automobile sales ratio',
 'shc_car_service_rt' : 'automobile service ratio',
 'shc_oil_rt': 'fuel ratio',
 'shc_e_comm_rt' : 'e-commerce ratio',
 'shc_dlv_rt' : 'food delivery app ratio',
 'shc_hotel_rt' : 'luxury hotel ratio',
 'shc_jj_rt' : 'Jeju island ratio',
 'shc_travel_os_rt' : 'oversea travel ratio',
 'shc_starbucks_rt' : 'starbucks ratio',
 'shc_e_charge_rt' : 'electronic vehicle charging ratio',
 'shc_tesla_charge_rt' : 'Tesla charging ratio',
 'shc_conv_rt' : 'convenience store ratio'}


ds_combined.rename(columns = new_var_name, inplace = True)

In [ ]:
ds_combined = ds_combined[ds_combined['months_from_birth'].between(-12,12)]

In [ ]:
control_var = ['ICM','TOT_ASST']
add_var = [col for col in ds_combined if col.endswith('amt')]

agg_dict =   {col: 'sum' for col in ['Card Spending AMT', 'Credit Card Spending AMT','Debit Card Spending AMT',
                  'Lump-sum Payment AMT', 'Installment Payment AMT', 'Cash Advance AMT','Overseas Card Spending AMT',]
                  +add_var
                  # 'weekday spending ratio','food industry ratio','entertainment ratio','dep store ratio',
                  #   'large store ratio','small retail ratio','clothing/accessories ratio','sport/culture/leiture ratio',
                  #   'accomodation ratio','travel ratio','transportation ratio','beauty ratio','household living ratio',
                  #   'education ratio','medical ratio','electronics/furniture ratio','automobile sales ratio',
                  #   'automobile service ratio', 'fuel ratio', 'e-commerce ratio', 'food delivery app ratio', 'luxury hotel ratio',
                  #   'oversea travel ratio',
                   + control_var}



agg_dict['ICM quantile'] = 'max'
agg_dict['TOT_ASST quantile'] = 'max'
agg_dict['months_from_birth'] = 'max'
agg_dict['birth event'] = 'max'



df_hshd = (
    ds_combined
    .groupby(['HSHD_SEQNO', 'BS_YR_MON'])
    .agg(agg_dict)
    .reset_index()
)

df_hshd.sort_values(['HSHD_SEQNO','BS_YR_MON'],inplace=True)
df_hshd['months_from_birth'] = df_hshd['months_from_birth'].astype(int)

In [ ]:
# double counting 우려 때문에 배달앱, 특급호텔, 제주 여행 등은 포함하지 않았음
category = ['shc_food_amt', 'shc_ent_amt','shc_dep_amt','shc_mart_amt', 'shc_ssm_amt','shc_clothes_amt',
    'shc_cul_amt','shc_acco_amt','shc_travel_amt','shc_trans_amt','shc_beauty_amt','shc_household_amt',
    'shc_edu_amt','shc_med_amt','shc_furn_amt','shc_car_amt','shc_car_service_amt','shc_oil_amt','shc_e_comm_amt']

df_hshd['total spending'] = df_hshd[category].sum(axis=1)

In [ ]:
amt_col = [col for col in df_hshd if col.endswith('amt')]
rt_cols = [c.replace('amt', 'rt') for c in amt_col]
df_hshd[rt_cols] = df_hshd[amt_col].div(df_hshd['total spending'], axis=0)
df_hshd.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df_hshd = pd.get_dummies(df_hshd, columns=['months_from_birth'], prefix='e', drop_first = False, dtype=int)
df_hshd = df_hshd.set_index(['HSHD_SEQNO','BS_YR_MON'])
df_hshd.drop(columns=['e_-1'], inplace=True)

In [ ]:
var_interested = ['Card Spending AMT', 'Credit Card Spending AMT','Debit Card Spending AMT',
                    'Lump-sum Payment AMT', 'Installment Payment AMT', 'Cash Advance AMT','Overseas Card Spending AMT',
                  'weekday spending ratio','food industry ratio','entertainment ratio','dep store ratio',
                    'large store ratio','small retail ratio','clothing/accessories ratio','sport/culture/leiture ratio',
                    'accomodation ratio','travel ratio','transportation ratio','beauty ratio','household living ratio',
                    'education ratio','medical ratio','electronics/furniture ratio','automobile sales ratio',
                    'automobile service ratio', 'fuel ratio', 'e-commerce ratio', 'food delivery app ratio', 'luxury hotel ratio',
                    'oversea travel ratio',
                  ]

Aggregate Level

In [ ]:
event_cols = [c for c in df_hshd.columns if c.startswith('e_')]
X = df_hshd[event_cols + control_var]
output_dir = '/mnt/sda1/RA5/output/siyoung/event_study_birth'

In [ ]:
for var in var_interested:

    try :
        y = df_hshd[var]
        model = PanelOLS(y, X, entity_effects=True, time_effects=True)
        res = model.fit(cov_type="clustered", cluster_entity=True)


        coefs = res.params[event_cols]
        ses = res.std_errors[event_cols]

        event_cols = [c for c in res.params.index if re.match(r"^e_-?\d+$", c)]
        event_times = [int(c.split("_")[1]) for c in event_cols]

        # Build df_plot with names preserved as index
        df_plot = pd.DataFrame({
            "event_time": event_times,
            "coef": res.params.loc[event_cols].values,
            "se": res.std_errors.loc[event_cols].values
        }, index=event_cols).sort_values("event_time")

        # Identify lead variables by NAME, not by row number
        lead_vars = [c for c in df_plot.index
                    if df_plot.loc[c, "event_time"] < 0 and df_plot.loc[c, "event_time"] != -1]

        if lead_vars:
            b = res.params.loc[lead_vars].values
            V = res.cov.loc[lead_vars, lead_vars].values
            W = float(b.T @ pinv(V) @ b)
            m = len(b)
            Fstat = W / m
            df1, df2 = m, res.df_resid
            pval = 1 - f.cdf(Fstat, df1, df2)
            pretrend_text = f"Pretrend: F({df1},{df2:.0f})={Fstat:.2f}, p={pval:.3f}"
        else:
            pretrend_text = "Pretrend: (no leads to test)"



        # Insert baseline point
        df_plot = pd.concat([
            df_plot,
            pd.DataFrame({'event_time': [-1], 'coef': [0], 'se': [0]})
        ])

        df_plot = df_plot.sort_values('event_time')

        # Plot with baseline included
        plt.errorbar(df_plot['event_time'], df_plot['coef'], yerr=1.96*df_plot['se'], fmt='o-')
        plt.axhline(0, color='gray', linestyle='--')

        plt.xlabel("Months from birth")
        plt.ylabel("Effect on y")
        plt.title(f'{var}')
        plt.tight_layout()
        if '/' in var:
            var = var.replace('/','_')
        save_path = os.path.join(output_dir, f"{var}_aggregate.png")
        plt.savefig(save_path, dpi=300)

        plt.show()
        print(pretrend_text)

        try:
            print(pretrend_text)
        except Exception:
            pass

    except Exception as e:
        print(f"Error happend {var}: {e}")

Heterogeneity

In [ ]:
for var in var_interested:

    for het in ['ICM quantile', 'TOT_ASST quantile']:

        try:
            quantiles = np.sort(df_hshd[het].dropna().unique())

            n = len(quantiles)
            ncols = 5
            nrows = int(np.ceil(n / ncols))

            fig, axes = plt.subplots(nrows, ncols, figsize=(20, 4*nrows), sharey=True)

            axes = axes.flatten()

            pretrend = []

            for i, q in enumerate(quantiles):

                df = df_hshd[df_hshd[het] == q]
                X = df[event_cols]
                y = df[var]
                if "ratio" not in var.lower():
                    log_value = True
                    y = np.log1p(y + 1)
                else:
                    log_value = False


                model = PanelOLS(y, X, entity_effects=True, time_effects=True)
                res = model.fit(cov_type="clustered", cluster_entity=True)

                coefs = res.params
                ses = res.std_errors

                event_cols = [c for c in res.params.index if re.match(r"^e_-?\d+$", c)]
                event_times = [int(c.split("_")[1]) for c in event_cols]

                # Build df_plot with names preserved as index
                df_plot = pd.DataFrame({
                    "event_time": event_times,
                    "coef": res.params.loc[event_cols].values,
                    "se": res.std_errors.loc[event_cols].values
                }, index=event_cols).sort_values("event_time")

                # Identify lead variables by NAME, not by row number
                lead_vars = [c for c in df_plot.index
                            if df_plot.loc[c, "event_time"] < 0 and df_plot.loc[c, "event_time"] != -1]

                if lead_vars:
                    b = res.params.loc[lead_vars].values
                    V = res.cov.loc[lead_vars, lead_vars].values
                    W = float(b.T @ pinv(V) @ b)
                    m = len(b)
                    Fstat = W / m
                    df1, df2 = m, res.df_resid
                    pval = 1 - f.cdf(Fstat, df1, df2)
                    pretrend_text = f"Pretrend: F({df1},{df2:.0f})={Fstat:.2f}, p={pval:.3f}"
                else:
                    pretrend_text = "Pretrend: (no leads to test)"

                pretrend.append(pretrend_text)


                # Insert baseline point
                df_plot = pd.concat([
                    df_plot,
                    pd.DataFrame({'event_time': [-1], 'coef': [0], 'se': [0]})
                ])

                df_plot = df_plot.sort_values('event_time')

                ax = axes[i]
                ax.errorbar(df_plot['event_time'], df_plot['coef'],
                            yerr=1.96*df_plot['se'], fmt='o-')
                ax.axhline(0, color='gray', linestyle='--')
                ax.set_title(f"Q: {int(q)}")
                ax.set_xlabel("Months from birth")
                if i == 0:
                    if log_value:
                        ax.set_ylabel("Effect on log y")

                    else:
                        ax.set_ylabel("Effect on y")



            for j in range(i+1, len(axes)):
                fig.delaxes(axes[j])

            fig.suptitle(f"Event Study for {var} by {het}", fontsize=16, y=1.02)
            plt.tight_layout()

            try:
                for q,text in enumerate(pretrend):
                    print(f"Q{q+1}:",pretrend_text)

            except Exception:
                pass

            if '/' in var:
                var = var.replace('/','_')

            save_path = os.path.join(output_dir, f"{var}_by_{het}.png")
            plt.savefig(save_path, dpi=300)

            plt.show()
        except Exception as e:
            print(f"Error happend {var} for {het}: {e}")


In [1]:
# For individual level

In [ ]:
ds_combined['months_from_birth'] = ds_combined['months_from_birth'].astype(int)
category = ['shc_food_amt', 'shc_ent_amt','shc_dep_amt','shc_mart_amt', 'shc_ssm_amt','shc_clothes_amt',
    'shc_cul_amt','shc_acco_amt','shc_travel_amt','shc_trans_amt','shc_beauty_amt','shc_household_amt',
    'shc_edu_amt','shc_med_amt','shc_furn_amt','shc_car_amt','shc_car_service_amt','shc_oil_amt','shc_e_comm_amt']

ds_combined['total spending'] = ds_combined[category].sum(axis=1)
amt_col = [col for col in ds_combined if col.endswith('amt')]
rt_cols = [c.replace('amt', 'rt') for c in amt_col]
ds_combined[rt_cols] = ds_combined[amt_col].div(ds_combined['total spending'], axis=0)
ds_combined.replace([np.inf, -np.inf], np.nan, inplace=True)


ds_combined = pd.get_dummies(ds_combined, columns=['months_from_birth'], prefix='e', drop_first = False, dtype=int)
ds_combined = ds_combined.set_index(['KEY','BS_YR_MON'])
ds_combined.drop(columns=['e_-1'], inplace=True)
event_cols = [c for c in ds_combined.columns if c.startswith('e_')]

In [ ]:
for var in var_interested:

    for het in ['sns_quantile', 'portal_quantile', 'region']:

        quantiles = np.sort(ds_combined[het].dropna().unique())

        n = len(quantiles)
        ncols = min(n, 5)
        nrows = int(np.ceil(n / ncols))

        fig, axes = plt.subplots(nrows, ncols, figsize=(20, 4*nrows), sharey=True)

        axes = axes.flatten()

        pretrend = []

        for i, q in enumerate(quantiles):

            df = ds_combined[ds_combined[het] == q]
            X = df[event_cols]
            y = df[var]
            if "ratio" not in var.lower():
                log_value = True
                y = np.log1p(y + 1)
            else:
                log_value = False


            model = PanelOLS(y, X, entity_effects=True, time_effects=True)
            res = model.fit(cov_type="clustered", cluster_entity=True)

            coefs = res.params
            ses = res.std_errors

            event_cols = [c for c in res.params.index if re.match(r"^e_-?\d+$", c)]
            event_times = [int(c.split("_")[1]) for c in event_cols]

            # Build df_plot with names preserved as index
            df_plot = pd.DataFrame({
                "event_time": event_times,
                "coef": res.params.loc[event_cols].values,
                "se": res.std_errors.loc[event_cols].values
            }, index=event_cols).sort_values("event_time")

            # Identify lead variables by NAME, not by row number
            lead_vars = [c for c in df_plot.index
                        if df_plot.loc[c, "event_time"] < 0 and df_plot.loc[c, "event_time"] != -1]

            if lead_vars:
                b = res.params.loc[lead_vars].values
                V = res.cov.loc[lead_vars, lead_vars].values
                W = float(b.T @ pinv(V) @ b)
                m = len(b)
                Fstat = W / m
                df1, df2 = m, res.df_resid
                pval = 1 - f.cdf(Fstat, df1, df2)
                pretrend_text = f"Pretrend: F({df1},{df2:.0f})={Fstat:.2f}, p={pval:.3f}"
            else:
                pretrend_text = "Pretrend: (no leads to test)"

            pretrend.append(pretrend_text)


            # Insert baseline point
            df_plot = pd.concat([
                df_plot,
                pd.DataFrame({'event_time': [-1], 'coef': [0], 'se': [0]})
            ])

            df_plot = df_plot.sort_values('event_time')

            ax = axes[i]
            ax.errorbar(df_plot['event_time'], df_plot['coef'],
                        yerr=1.96*df_plot['se'], fmt='o-')
            ax.axhline(0, color='gray', linestyle='--')
            ax.set_title(f"Q: {int(q)}")
            ax.set_xlabel("Months from birth")
            if i == 0:
                if log_value:
                    ax.set_ylabel("Effect on log y")

                else:
                    ax.set_ylabel("Effect on y")



        for j in range(i+1, len(axes)):
            fig.delaxes(axes[j])

        fig.suptitle(f"Event Study for {var} by {het}", fontsize=16, y=1.02)
        plt.tight_layout()

        try:
            for q,text in enumerate(pretrend):
                print(f"Q{q+1}:",pretrend_text)

        except Exception:
            pass

        if '/' in var:
            var = var.replace('/','_')

        save_path = os.path.join(output_dir, f"{var}_by_{het}.png")
        plt.savefig(save_path, dpi=300)

        plt.show()